In [1]:
from tennis_model_predict import get_model_win_proba

from tennis_model_predict import general_elo_dict, hard_dict, ts_dict

from elo_funcs import elo_predict

C:\Users\jyoung\Projects\tennis_project\tennis_model_predict.py:54: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_folder}/processed_apt_with_ts.csv', parse_dates=['tourney_date'])


In [15]:
get_model_win_proba(player_1='Constant Lestienne',
                    player_2='Emil Ruusuvuori',
                    surface='Hard',
                    tourney_level='C',
                    round_=5)

0.34002927

In [54]:
for x in ts_dict.keys():
    if 'Ramanathan' in x:
        print(x)

Ramkumar Ramanathan


In [ ]:
tourney_surface = 'Hard'
tourney_level = 'A'
tourney_round = 4

player_pairs = []
probas = []

for pair in player_pairs:
    proba = get_model_win_proba(player_1=pair[0],
                                player_2=pair[1],
                                surface=surface,
                                tourney_level=tourney_level,
                                round_=tourney_round)
    probas.append(proba)


In [13]:
general_elo_dict

NameError: name 'general_elo_dict' is not defined

In [20]:
elo_predict(hard_dict['Dominic Thiem'], hard_dict['Marin Cilic'])

0.671245029231464

In [33]:
hard_dict['Dominic Thiem']

2359.1754152087015

In [34]:
hard_dict['Marin Cilic']

2235.171913381185

In [37]:
ts_dict['Dominic Thiem'][-1]

(571559, N(mu=3.096, sigma=0.254))

In [38]:
ts_dict['Marin Cilic'][-1]

(571668, N(mu=3.244, sigma=0.258))